In [ ]:
kernel.silent(true)

In [1]:
import coursierapi.MavenRepository
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

import coursierapi.MavenRepository


In [2]:
import $ivy.`com.github.propi:rdfrules:1.5.0`
import collection._
import org.apache.jena.riot.Lang
import reflect.io._, Path._
import reflect.io._
import Path._
import com.github.propi.rdfrules.data._
import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._
import com.github.propi.rdfrules.ruleset._
kernel.silent(false)

import $ivy.$                                

import collection._

import org.apache.jena.riot.Lang

import reflect.io._, Path._

import reflect.io._

import Path._

import com.github.propi.rdfrules.data._

import com.github.propi.rdfrules.algorithm.amie._

import com.github.propi.rdfrules.algorithm.dbscan._

import com.github.propi.rdfrules.utils._

import com.github.propi.rdfrules.index._

import com.github.propi.rdfrules.rule._

import com.github.propi.rdfrules.ruleset._


In [3]:
val uri = (value: String) => TripleItem.Uri(value)
val qbDataSet = "http://purl.org/linked-data/cube#dataSet";

uri: String => TripleItem.Uri = ammonite.$sess.cmd2$Helper$$Lambda$2824/0x0000000840c77840@4763f014
qbDataSet: String = "http://purl.org/linked-data/cube#dataSet"

In [4]:
val graphs = "../data/jaur".toDirectory.files.map(_.path).filter(g => g matches """.*\.ttl""")

graphs: Iterator[String] = non-empty iterator

In [5]:
var jaurCubesMap: Map[String, Graph] = Map()
graphs.foreach(g => {
    val name = g.replaceAll("\\.ttl", "").replaceAll("^.*/", "")
    jaurCubesMap = jaurCubesMap + (name -> Graph("czso",g))
})

jaurCubesMap: Map[String, Graph] = Map(
  "jaur-regions-bysex" -> com.github.propi.rdfrules.data.Graph@5cc09b6e,
  "jaur-regions-total" -> com.github.propi.rdfrules.data.Graph@563f64e0,
  "jaur-districts-total" -> com.github.propi.rdfrules.data.Graph@3ce41958,
  "jaur-districts-bysex" -> com.github.propi.rdfrules.data.Graph@6377dcdc
)

In [6]:
jaurCubesMap.foreach(r => println(r._1 + ": " + r._2.filter(t => t.predicate.hasSameUriAs(qbDataSet)).size))

jaur-regions-bysex: 234
jaur-regions-total: 117
jaur-districts-total: 684
jaur-districts-bysex: 1368


In [ ]:
val intervalCounts = Array(5,10,20)



In [7]:
import eu.easyminer.discretization.impl.Interval
import eu.easyminer.discretization.impl.IntervalBound._
val rounded = (value: Double, scale: Integer) => BigDecimal(value).setScale(scale, BigDecimal.RoundingMode.HALF_UP).toDouble
val intervalToString = (i: Interval) => "<"+rounded(i.minValue.value,2)+ "__"+rounded(i.maxValue.value,2)+")"

import eu.easyminer.discretization.impl.Interval

import eu.easyminer.discretization.impl.IntervalBound._

rounded: (Double, Integer) => Double = ammonite.$sess.cmd6$Helper$$Lambda$3040/0x0000000840dfa040@6c411ee
intervalToString: Interval => String = ammonite.$sess.cmd6$Helper$$Lambda$3041/0x0000000840dfa840@45f5202f

In [8]:
// in all cubes
val czsoUri = "http://data.czso.cz/ontology/"
val unemploymentRate = uri(czsoUri+"podilNezamestnanych")
val reachableApplicants = uri(czsoUri+"dosazitelniNeumisteniUchazeciOZamestnani")

// only in total cubes
val unplacedApplicants = uri(czsoUri+"neumisteniUchazeciOZamestnani")
val vacaniesCount = uri(czsoUri+"pocetVolnychMist")

val measureUris = Array(unemploymentRate,reachableApplicants,unplacedApplicants,vacaniesCount) 

czsoUri: String = "http://data.czso.cz/ontology/"
unemploymentRate: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/podilNezamestnanych"
)
reachableApplicants: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/dosazitelniNeumisteniUchazeciOZamestnani"
)
unplacedApplicants: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/neumisteniUchazeciOZamestnani"
)
vacaniesCount: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/pocetVolnychMist"
)
measureUris: Array[TripleItem.Uri] = Array(
  LongUri("http://data.czso.cz/ontology/podilNezamestnanych"),
  LongUri(
    "http://data.czso.cz/ontology/dosazitelniNeumisteniUchazeciOZamestnani"
  ),
  LongUri("http://data.czso.cz/ontology/neumisteniUchazeciOZamestnani"),
  LongUri("http://data.czso.cz/ontology/pocetVolnychMist")
)

In [9]:
val equiFrequent = (intervalCount: Int) => DiscretizationTask.Equifrequency(intervalCount)


val discretizeEquifrequent = (graph: Graph, intervalsCount: Int, measure: TripleItem.Uri) => {
    graph.filter(t => t.predicate == measure)
         .discretize(equiFrequent(intervalsCount))(quad => quad.triple.predicate == measure)
}

val discretizeEquifrequentAndGetIntervals = (graph: Graph, intervalsCount: Int, measure: TripleItem.Uri) => {
    graph.filter(t => t.predicate == measure)
         .discretizeAndGetIntervals(equiFrequent(intervalsCount))(quad => quad.triple.predicate == measure)
}

equiFrequent: Int => DiscretizationTask.Equifrequency = ammonite.$sess.cmd8$Helper$$Lambda$3098/0x0000000840e1c840@50e698fa
discretizeEquifrequent: (Graph, Int, TripleItem.Uri) => Graph = ammonite.$sess.cmd8$Helper$$Lambda$3099/0x0000000840e1c440@78ea069
discretizeEquifrequentAndGetIntervals: (Graph, Int, TripleItem.Uri) => IndexedSeq[Interval] = ammonite.$sess.cmd8$Helper$$Lambda$3100/0x0000000840e1d040@15197a17

In [10]:
import eu.easyminer.discretization.impl.Interval

val renameInterval = (t: Triple, intervals: IndexedSeq[Interval], suffix: String) => {
    var result = t
    intervals.foreach(interval => {
        val _interval = t.`object`.asInstanceOf[TripleItem.Interval]
        if (_interval.interval == interval) {
            result = t.copy(`object` = TripleItem.Uri(_interval.toString+"_"+suffix+"_"+(intervals.indexOf(interval)+1)+"/"+intervals.size) )
        }
    })
    result
}

val discretize = (graph: Graph, name: String, intervalCounts: Array[Int]) => {
    var result = Dataset() + 
    graph.filter(t => !measureUris.contains(t.predicate))
    .map(t => if (t.predicate.hasSameUriAs(qbDataSet)) t.copy(`object` = uri(name)) else t)
    
        measureUris.foreach(measure => {
            intervalCounts.foreach(intervalCount => {
                val intervals = discretizeEquifrequentAndGetIntervals(graph, intervalCount, measure)
                val measures = discretizeEquifrequent(graph, intervalCount, measure)
                result = result + measures.map(t => {renameInterval(t, intervals, "ef"+intervalCount)})
            })
    })
    result
}

import eu.easyminer.discretization.impl.Interval


renameInterval: (Triple, IndexedSeq[Interval], String) => Triple = ammonite.$sess.cmd9$Helper$$Lambda$3140/0x0000000840e3d840@781ad66b
discretize: (Graph, String, Array[Int]) => Dataset = ammonite.$sess.cmd9$Helper$$Lambda$3141/0x0000000840e3dc40@35404fd3

In [10]:
jaurCubesMap.foreach(d => discretize(d._2,d._1).cache("../cache/jaur/"+d._1+".cache").export("../exports/jaur/"+d._1+".ttl"))

cmd10.sc:1: not enough arguments for method apply: (v1: com.github.propi.rdfrules.data.Graph, v2: String, v3: Array[Int])com.github.propi.rdfrules.data.Dataset in trait Function3.
Unspecified value parameter v3.
val res10 = jaurCubesMap.foreach(d => discretize(d._2,d._1).cache("../cache/jaur/"+d._1+".cache").export("../exports/jaur/"+d._1+".ttl"))
                                                ^Compilation Failed

: 

In [16]:
discretize(jaurCubesMap.get("jaur-regions-bysex").get,"jaur-regions-bysex",Array(5,10))
.cache("../cache/jaur/jaur-regions-bysex.cache")
.export("../exports/jaur/jaur-regions-bysex.ttl")

discretize(jaurCubesMap.get("jaur-regions-total").get,"jaur-regions-total",Array(5,10))
.cache("../cache/jaur/jaur-regions-total.cache")
.export("../exports/jaur/jaur-regions-total.ttl")

discretize(jaurCubesMap.get("jaur-districts-bysex").get,"jaur-districts-bysex",Array(10,20,30))
.cache("../cache/jaur/jaur-districts-bysex.cache")
.export("../exports/jaur/jaur-districts-bysex.ttl")

discretize(jaurCubesMap.get("jaur-districts-total").get,"jaur-districts-total",Array(10,20,30))
.cache("../cache/jaur/jaur-districts-total.cache")
.export("../exports/jaur/jaur-districts-total.ttl")